In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import datetime

import os
print(os.listdir("../input"))

from sklearn.linear_model import LogisticRegression
from collections import defaultdict
# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir("../input/makejuneextradata-fff8dd"))


In [ ]:
start_time = datetime.datetime.now()

demographic_cols = ['fecha_dato',
 'ncodpers','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel',
 'indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall',
 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento']

notuse = ["ult_fec_cli_1t","nomprov"]

product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

train_cols = demographic_cols + product_col

In [ ]:
df_train = pd.read_csv('../input/makejuneextradata-fff8dd/juneExtra.csv', usecols=train_cols)

In [ ]:
df_test = pd.read_csv('../input/santander-product-recommendation/test_ver2.csv', usecols = demographic_cols)

In [ ]:
# The demographic features of each customer in june 2016
df_test.head()

Then , I removed all the rows that contains outliers values

In [ ]:
df_train = df_train[df_train['ind_nuevo'] == 0]
df_train = df_train[df_train['antiguedad'] != -999999]
df_train = df_train[df_train['indrel'] == 1]
df_train = df_train[df_train['indresi'] == 'S']
df_train = df_train[df_train['indfall'] == 'N']
df_train = df_train[df_train['tipodom'] == 1]
df_train = df_train[df_train['ind_empleado'] == 'N']
df_train = df_train[df_train['pais_residencia'] == 'ES']
df_train = df_train[df_train['indrel_1mes'] == 1]
df_train = df_train[df_train['tiprel_1mes'] == ('A' or 'I')]
df_train = df_train[df_train['indext'] == 'N']

After that,  I dropped unused columns that I don't need it

In [ ]:
drop_column = ['ind_nuevo','indrel','indresi','indfall','tipodom','ind_empleado','pais_residencia','indrel_1mes','indext','conyuemp','fecha_alta','tiprel_1mes']

df_train.drop(drop_column, axis=1, inplace = True)
df_test.drop(drop_column, axis=1, inplace = True)

Transfering the column "renta" (income) from object to numeric

In [ ]:
df_train["renta"]   = pd.to_numeric(df_train["renta"], errors="coerce")
df_test["renta"]   = pd.to_numeric(df_test["renta"], errors="coerce")

unique_prov: contains all cities ids

In [ ]:
unique_prov = df_test[df_test.cod_prov.notnull()].cod_prov.unique()

impute the missing values of some customers with the mean values of the others customers in the same city

In [ ]:
grouped = df_test.groupby("cod_prov")["renta"].median()

In [ ]:
for cod in unique_prov:
    df_test.loc[df_test['cod_prov']==cod,['renta']] = df_test.loc[df_test['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values
    df_train.loc[df_train['cod_prov']==cod,['renta']] = df_train.loc[df_train['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values

In [ ]:
df_test.renta.fillna(df_test["renta"].median(), inplace=True)

In [ ]:
# not in use
replace_mapping = {
                  "sexo": {"V":0,'H':1},
                  }

df_train.replace(replace_mapping, inplace= True)

In [ ]:
# These column are categories feature, I'll transform them using get_dummy
dummy_col = ['sexo','canal_entrada','cod_prov','segmento']

In [ ]:
limit = int(0.05 * len(df_train.index))

for col in dummy_col:
    if len(df_train[col].unique()) > 6:
        trainlist = df_train[col].value_counts()
        print(trainlist)
        use_col = []
        for i,item in enumerate(trainlist):
            if item > limit:
                use_col.append(df_train[col].value_counts().index[i])  
        for item in df_train[col].unique(): 
            if item not in use_col:
                row_index = df_train[col] == item
                df_train.loc[row_index,col] = np.nan
        for item in df_test[col].unique(): 
            if item not in use_col:
                row_index = df_test[col] == item
                df_test.loc[row_index,col] = np.nan 

In [ ]:
df_train = pd.get_dummies(df_train, prefix=dummy_col, columns = dummy_col)
df_test = pd.get_dummies(df_test, prefix=dummy_col, columns = dummy_col)

In [ ]:
df_train.head()

In [ ]:
df_train["age"]   = pd.to_numeric(df_train["age"], errors="coerce")
df_test["age"]   = pd.to_numeric(df_test["age"], errors="coerce")

age_group = [[0,19],[19,26],[26,36],[36,41],[41,47],[47,55],[55,60],[60,70],[70,80],[80,170]]

def create_age_group(df):  
    df['age_group'] = np.nan
    for i,age in enumerate(age_group):
        row_index = (df['age'] >= age[0]) & (df['age'] < age[1])
        df.loc[row_index,'age_group'] = i
        
create_age_group(df_train)
create_age_group(df_test)

df_train.drop('age', axis=1, inplace = True)
df_test.drop('age', axis=1, inplace = True)

In [ ]:
df_train.head()

In [ ]:
renta_group = [[0,50000],[50000,70000],[70000,100000],[100000,150000],[150000,200000],[200000,1000000],[1000000,29000000]]

def create_renta_group(df):  
    df['renta_group'] = np.nan
    for i,renta in enumerate(renta_group):
        row_index = (df['renta'] >= renta[0]) & (df['renta'] < renta[1])
        df.loc[row_index,'renta_group'] = i
        
create_renta_group(df_train)
create_renta_group(df_test)

df_train.drop('renta', axis=1, inplace = True)
df_test.drop('renta', axis=1, inplace = True)

In [ ]:
df_train.head()

In [ ]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = df_test['ncodpers'].values
for c in product_col:
    if c != 'ncodpers':
        print(c)
        # train model here with june 2015 data
        y_train = df_train[c]
        x_train = df_train.drop(product_col + ['ncodpers',"fecha_dato"], 1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        
        # predict model with the most recent data

        x_train2 = df_test.drop(['ncodpers',"fecha_dato"], 1)
        p_train = clf.predict_proba(x_train2)[:,1]
        p_train2 = clf.predict_proba(x_train)[:,1]

        models[c] = clf
        model_preds[c] = p_train
        
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
                    

In [ ]:
#the probability of purchase for each customer for the last product
list(p_train)

In [ ]:
df_recent = pd.read_csv('../input/santander-product-recommendation/train_ver2.csv',usecols=['ncodpers'] + product_col)
df_recent = df_recent.drop_duplicates(['ncodpers'], keep='last')

In [ ]:
df_recent = pd.read_csv('../input/santander-product-recommendation/train_ver2.csv',usecols=['ncodpers'] + product_col)


In [ ]:
df_recent.head()

In [ ]:
sample = pd.read_csv('../input/santander-product-recommendation/sample_submission.csv')

In [ ]:
# check if customer already have each product or not. 
already_active = {}
for row in df_recent.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(tuple(product_col), row) if c[1] > 0]
    already_active[id] = active

# add 7 products(that user don't have yet), higher probability first -> train_pred   
train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    preds = [i[0] for i in sorted([i for i in zip(tuple(product_col), p) if i[0] not in already_active[id]],
                                  key=lambda i:i [1], 
                                  reverse=True)[:7]]
    train_preds[id] = preds
    
test_preds = []
for row in sample.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))


In [ ]:
already_active

Display the 7  products that have the higher probabilty of purchases for each customer that he doesn't have yet

In [ ]:
train_preds

In [ ]:
sample['added_products'] = test_preds


In [ ]:
sample.head()

In [ ]:
sample.to_csv('LR_sub.csv', index=False)
print(datetime.datetime.now()-start_time)
